In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
def train_data(N_x, x_l, x_r, N_t, t_i, t_f, del_t, sel_x):

    x_train = np.ones((N_x,1,1))
    for i in range(N_x):
        x_train[i][0][0] = t_i + del_x*i
        
    t_train = np.ones((N_x,1,1))
    null = np.zeros((N_x,1,1))
    x_train = torch.FloatTensor(x_train)
    x_train = x_train.clone().detach().requires_grad_(True)
    t_train = torch.FloatTensor(t_train)
    null = torch.FloatTensor(null)
    null = null.clone().detach().requires_grad_(True)
    return x_train, t_train, null

def ini_temp(N_x, T_r, T_l):
    u_train = np.zeros(N_x)
    u_train[0] = T_l
    u_train[-1] = T_r
    u_train = torch.FloatTensor(u_train)
    u_train = u_train.unsqueeze(1)
    u_train = u_train.clone().detach().requires_grad_(True)
    return u_train
    
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, rnn_size, rnn_layers, hidden_layers):
        super(SimpleRNN, self).__init__()
        
        # RNN model
        self.rnn = nn.GRU(input_size, rnn_size, rnn_layers)
        
        # Fully conected model
        modules = []
        for i in range(hidden_layers):
            if(i == 0):
                modules.append(nn.Linear(rnn_size, hidden_size))
            else:
                modules.append(nn.Linear(hidden_size, hidden_size)) 
            modules.append(nn.Tanh())
            
        if(hidden_layers == 0):
            hidden_size = rnn_size 
            
        modules.append(nn.Linear(hidden_size, output_size))
        modules.append(nn.Tanh())
        self.fc = nn.Sequential(*modules)
        
        # initialise the weights
        for layer in self.fc.modules():
            if isinstance(layer, nn.Linear):
                layer.weight.data.normal_(mean=0, std=0.2)

        for layer in range(rnn_layers):
            for weight in self.rnn._all_weights[layer]:       
                if "weight" in weight:
                    nn.init.xavier_normal_(getattr(self.rnn,weight))

    def forward(self, x_train, t_train, t_i, N_t, del_t, N_x):
        
        self.h0 = torch.zeros(rnn_layers, 1, rnn_size)
        
        T = torch.zeros((N_t, N_x, 1))
        T_xx = torch.zeros((N_t, N_x, 1))
        T_t = torch.zeros((N_t, N_x, 1))
        for i in range(N_t):
            t_train2 = t_train*(t_i + i*del_t)
            t_train3 = t_train2.clone().detach().requires_grad_(True)
            X = torch.cat((x_train, t_train3), 2)
            rnn_output, hidden = self.rnn(X, self.h0)
#             print(t_train3)
            
            op = self.fc( rnn_output[:, -1, :])
            op_x = torch.autograd.grad(op, x_train, grad_outputs=torch.ones_like(op), create_graph=True)[0]
            op_t = torch.autograd.grad(op, t_train3, grad_outputs=torch.ones_like(op), create_graph=True)[0]
            op_xx = torch.autograd.grad(op_x, x_train, grad_outputs=torch.ones_like(op_x), create_graph=True)[0]
            op_t = torch.squeeze(op_t, 2)
            op_xx = torch.squeeze(op_xx, 2)
            
            self.h0 = hidden
            T[i] = op
            T_xx[i] = op_xx
            T_t[i] = op_t
            
        return T, T_xx, T_t, x_train, t_train
            
def train_model(model, optimiser, epochs, T_ini, null, N_t, T_l, T_r, x_l, x_r, c1, x_train, t_train, N_x):
    loss_str = []
    iters = []
    mse = nn.MSELoss()
    model.train()  # Set the model in training model
    for epoch in range(epochs):
#         print(epoch)
        T, T_xx, T_t, x_train2, t_train2 = model(x_train, t_train, t_i, N_t, del_t, N_x)
        optimiser.zero_grad() 
        
        eq = 0
        bc1 = 0
        bc2 = 0
        ic = 0
        for i in range(N_t):
            eq += mse(T_t[i] , c1*T_xx[i])
            bc1 += mse( torch.mul(torch.where(x_train2 == x_l,1,0),(T[i] - T_l)), null ) 
            bc2 += mse( torch.mul(torch.where(x_train2 == x_r,1,0),(T[i] - T_r)), null ) 
            ic += mse( torch.mul(torch.where(t_train2 == t_i,1,0),(T[i] - T_ini)), null) 
                
        loss = (eq + bc1 + bc2 + ic)/N_t
        loss.backward()   
        optimiser.step()
        loss_str.append(loss.detach().numpy())
        iters.append(epoch)
        
        if epoch%10 == 0:
            print('epoch = ',epoch,', loss = ',loss.detach().numpy())
            
    return iters, loss_str
        
        

In [3]:
# Training data parameters and B.C
N_x = 35
N_t = 35
x_l = 0
x_r = 1
t_i = 0
t_f = 0.8
del_t = (t_f - t_i)/(N_t - 1)
del_x = (x_r - x_l)/(N_x - 1)
T_l = 1.0
T_r = 0.0

# Neural network params
input_size = 2
rnn_size = 3
hidden_size = 6
output_size = 1
hidden_layers = 0
rnn_layers = 1

# Equation Parameters
c1 = 0.1

# Training Data
x_train, t_train, null = train_data(N_x, x_l, x_r, N_t, t_i, t_f, del_t, del_x)
T_ini = ini_temp(N_x, T_r, T_l)
model = SimpleRNN(input_size, hidden_size, output_size, rnn_size, rnn_layers, hidden_layers)
print(model)
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable parameters in the model:", total_trainable_params)

# Training model
learning_rate = 2e-3
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 1500
iters, loss_str = train_model(model, optimiser, epochs, T_ini, null, N_t, T_l, T_r, x_l, x_r, c1, x_train, t_train, N_x)

SimpleRNN(
  (rnn): GRU(2, 3)
  (fc): Sequential(
    (0): Linear(in_features=3, out_features=1, bias=True)
    (1): Tanh()
  )
)
Total trainable parameters in the model: 67


c:\users\sanjeet\appdata\local\programs\python\python39\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([41, 1, 1])) that is different to the input size (torch.Size([41, 41, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch =  0 , loss =  0.032342635
epoch =  10 , loss =  0.022675488


KeyboardInterrupt: 